In [2]:
from together import Together
import json
from tqdm import tqdm
import csv
import time

In [3]:
ann = json.load(open(f'Annotation.json'))

In [3]:
reader = csv.reader(open('image_urls.csv'))
urls = list(reader)

In [4]:
data = []
for i in tqdm(range(len(ann))):
    data.append({
        'question': ann[i]['question'],
        'answers': ann[i]['answers'],
        'image_url': urls[i][0]
    })

100%|██████████| 4319/4319 [00:00<00:00, 722354.21it/s]


In [16]:
client = Together(api_key="581f09912b905cd058e92cd71321e5385267b1b3c41396cf8d5057f4ee599bb2")

In [19]:
for i in tqdm(range(len(data))):
    while True:
        try:
            question = data[i]['question']
            prompt = f"You will be asked a question and you have to give a precise answer. If you don't know the answer, respond with 'unanswerable.'\nQuestion: {question}"
            imageUrl = data[i]['image_url']
            output = client.chat.completions.create(
                model="meta-llama/Llama-Vision-Free",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": imageUrl,
                                },
                            },
                        ],
                    }
                ],
                max_tokens=512,
                temperature=0.7,
                top_p=0.7,
                top_k=50,
                repetition_penalty=1,
                stop=["<|eot_id|>","<|eom_id|>"],
                stream=False
            )
            break
        except Exception as e:
            time.sleep(60)
    answer = output.choices[0].message.content
    data[i]['prediction'] = answer
    json.dump(data, open('Llama3_2_VizWiz_Val.json', 'w'), indent=4)

100%|██████████| 158/158 [22:46<00:00,  8.65s/it]
